### < Topic >

1. review


2. topic

    2.1 피드 포워드 신경망 언어 모델 (Neural Network Language Model, NNLM)

    2.2 순환 신경망 언어 모델 (Recurrent Neural Network Language Model, RNNLM)
    
    2.3 RNN 이용 텍스트 생성
    
    2.4 글자 단위 RNN (Char RNN)

## - 피드 포워드 신경망 언어 모델(Neural Network Language Model, NNLM)

- '언어 모델'은 문장에 확률을 할당하는 모델
- 주어진 문맥으로부터 아직 모르는 단어를 예측하는 것을 '언어 모델링'
- 이전 단어를 이용해서 다음 단어를 예측하는 언어 모델링 방법
- 통계적 언어 모델(Statistical Language Model, SLM): n-gram 언어 모델
- 신경망 언어 모델(Neural Network Language Mdel, NNLM): NNLM, RNN 언어 모델 

### n-gram 언어 모델

An adorable little boy is spreading [ 예측 단어 ]

- 4-gram 언어 모델

( An adorable little ) -> 무시

[ boy is spreading ] -> n-1개의 단어

P( [w] | boy is spreading ) = count( boy is spreading [w] ) / count( boy is spreading )

corpus -> boy is spreading : 1000번 출현 boy is spreading insults : 500번 출현 boy is spreading smiles : 200번 출현

P( [ insults ] | boy is spreading ) : 0.500  P( [ smiles ] | boy is spreading ) : 0.200

n-gram 단점 : 희소 문제( sparsity problem ) -> 언어 모델에서 학습하지 않는 단어에 대해서는 확률이 0이기 때문에 예측하지 못하는 문제가 있다.

- 희소 문제는 '단어의 유사도'를 알 수 있다면 해결할 수 있다.
- 언어 모델이 단어의 유사도를 학습할 수 있도록 설계한다면, corpus에 없는 단어 시퀀스에 대한 예측이라도 유사한 단어가 사용된 단어 시퀀스를 참고하여 보다 정확한 예측을 할 수 있을 것이다. -> NNLM

### 피드 포워드 신경망 언어 모델(NNLM) 

- NNLM도 n-gram LM과 유사하게 다음 단어를 예측할 때, 앞의 모든 단어를 참고하는 것이 아니라 정해진 n 개의 단어만을 참고
- (예측을 할 때 정해지는 n개의 범위: window)
- 충분한 양의 훈련 corpus를 NNLM 모델을 통해 학습한다면, 수많은 문장에서 유사한 목적으로 사용되는 단어들은 결국 유사한 임베딩 벡터값을 얻게 된다.

### RNN 언어 모델 (Recurrent Neural Network Language Model, RNNLM)

- n-gram 언어 모델이나 NNLM은 고정된 개수의 단어만을 입력으로 받아야 하는 단점
- 시점 (time step)이라는 개념이 도입된 RNN으로 언어 모델을 만들면 입력의 길이를 고정하지 않을 수 있다.

<img src = "https://wikidocs.net/images/page/46496/rnnlm1_final_final.PNG">

이전 시점의 출력 = 현재 시점의 입력

RNNLM의 학습: '교사 강요(teacher forcing)' 방법 - 테스트 과정에서 t 시점의 출력이 t+1 시점의 입력으로 사용되는 RNN 모델 훈련 기법
- t 시점에서 예측한 값을 t+1 시점에서 입력으로 사용하지 않고, t 시점의 레이블. 즉, 실제 알고있는 정답을 t+1 시점의 입력으로 사용
- 한 번 잘못 예측하면 뒤에서의 예측까지 영향을 미쳐 훈련 시간이 느려지게 되므로 교사 강요를 사용하여 RNN을 좀 더 빠르고 효과적으로 훈련

### RNN을 이용한 텍스트 생성

- 다대일(many-to-one) 구조와 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델

문장: '경마장에 있는 말이 뛰고 있다', '그의 말이 법이다', '가는 말이 고와야 오는 말이 곱다'

|samples|x|y|
|------|--|--|
|1|경마장에|있는|
|2|경마장에 있는|말이|
|3|경마장에 있는 말이|뛰고|
|4|경마장에 있는 말이 뛰고|있다|
|5|그의|말이
|6|그의 말이|법이다
|7|가는|말이
|8|가는 말이|고와야
|9|	가는 말이 고와야|오는
|10|가는 말이 고와야 오는|말이
|11|가는 말이 고와야 오는 말이|곱다

#### 1) 데이터 전처리

In [4]:
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [12]:
text = """경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고아야 오는 말이 곱다\n"""

In [13]:
text

'경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고아야 오는 말이 곱다\n'

In [17]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1  # keras 원-핫 인코딩에서 인덱스가 0부터 시작

print('단어 집합의 크기 : {}'.format(vocab_size))

단어 집합의 크기 : 12


In [20]:
# 정수 인코딩 결과

print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고아야': 9, '오는': 10, '곱다': 11}


In [21]:
# 훈련 데이터 생성

sequences = list()
for line in text.split('\n'):  # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: {:d}'.format(len(sequences)))

학습에 사용할 샘플의 개수: 11


In [22]:
# 레이블로 사용될 단어는 분리하지 않은 훈련 데이터

print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [24]:
# 가장 긴 샘플 길이를 기준으로 전체 샘플 길이 일치시켜야 함
# 가장 긴 샘플 길이 확인

max_len = max(len(l) for l in sequences)

print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [25]:
# 앞쪽에 패딩을 넣음 (길이 짧은 벡터는 0으로 채워짐)

sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')

In [26]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [27]:
# x, y 분리

sequences = np.array(sequences)
x = sequences[:,:-1] # 마지막 값을 제외
y = sequences[:,-1]  # 마지막 값만 저장

In [28]:
print(x)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [29]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [30]:
# 레이블을 원핫 인코딩

y = to_categorical(y, num_classes = vocab_size)

In [31]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


#### 2) RNN 모델

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [45]:
model = Sequential()

model.add(Embedding(vocab_size, 10, input_length = max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x, y, epochs = 200, verbose = 2)

Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.4720 - accuracy: 0.0909
Epoch 2/200
11/11 - 0s - loss: 2.4576 - accuracy: 0.1818
Epoch 3/200
11/11 - 0s - loss: 2.4432 - accuracy: 0.3636
Epoch 4/200
11/11 - 0s - loss: 2.4287 - accuracy: 0.3636
Epoch 5/200
11/11 - 0s - loss: 2.4141 - accuracy: 0.3636
Epoch 6/200
11/11 - 0s - loss: 2.3993 - accuracy: 0.5455
Epoch 7/200
11/11 - 0s - loss: 2.3842 - accuracy: 0.5455
Epoch 8/200
11/11 - 0s - loss: 2.3686 - accuracy: 0.5455
Epoch 9/200
11/11 - 0s - loss: 2.3525 - accuracy: 0.5455
Epoch 10/200
11/11 - 0s - loss: 2.3359 - accuracy: 0.4545
Epoch 11/200
11/11 - 0s - loss: 2.3186 - accuracy: 0.4545
Epoch 12/200
11/11 - 0s - loss: 2.3005 - accuracy: 0.4545
Epoch 13/200
11/11 - 0s - loss: 2.2817 - accuracy: 0.4545
Epoch 14/200
11/11 - 0s - loss: 2.2620 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.2413 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.2198 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 2.1973 - accuracy: 0.3636
Epo

Epoch 142/200
11/11 - 0s - loss: 0.2900 - accuracy: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.2836 - accuracy: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.2773 - accuracy: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2711 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2651 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2592 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2534 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2478 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2423 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2369 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2316 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2265 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2215 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.2166 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2119 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2072 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2027 - accuracy: 1.00

In [63]:
# 문장 생성 함수

def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재단어, 반복횟수(예측횟수)
    init_word = current_word   # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n):  # n번 반복
        encoded = t.texts_to_sequences([current_word])[0]           # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 x(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장
        
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어 인덱스와 동일한 단어가 있다면
                break           # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word         # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [65]:
print(sentence_generation(model, t, '경마장에', 4))

경마장에 있는 말이 뛰고 있다


In [66]:
print(sentence_generation(model, t, '경마장에', 10))

경마장에 있는 말이 뛰고 있다 말이 곱다 곱다 있다 곱다 곱다


In [59]:
print(sentence_generation(model, t, '그의', 2)) 

그의 말이 법이다


In [61]:
print(sentence_generation(model, t, '그의', 5)) 

그의 말이 법이다 오는 말이 곱다


### LSTM을 이용한 텍스트 생성

#### 1) 데이터 전처리

In [67]:
import pandas as pd
from string import punctuation

In [68]:
df = pd.read_csv('ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [69]:
# columns 확인

print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [71]:
# Null 값 확인

df['headline'].isnull().values.any()

False

In [72]:
# headline에서 신문 기사 제목을 뽑아 하나의 리스트로 저장

headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [74]:
# 전체 샘플 개수 확인

print('총 샘플의 개수: {}'.format(len(headline)))

총 샘플의 개수: 1324


In [75]:
# 데이터 중 'Unknown' 제거 필요

headline = [n for n in headline if n != 'Unknown']
print('노이즈값 제거 후 샘플의 개수: {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수: 1214


In [76]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [79]:
# 데이터 전처리: 구두점 제거, 단어의 소문자화

def repreprocessing(s):
    s = s.encode('utf-8').decode('ascii','ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

text = [repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [80]:
# 단어 집합 생성

t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: %d' % vocab_size)

단어 집합의 크기: 3494


In [81]:
# 훈련 데이터 생성

sequences = list()

for line in text:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [82]:
# 인덱스로부터 단어를 찾는 딕셔너리: index_to_word

index_to_word = {}
for key, value in t.word_index.items():  # key: 단어, value: 인덱스
    index_to_word[value] = key
    
print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : offer


In [83]:
# y 데이터 분리하기 전, 전체 샘플의 길이를 동일하게 만드는 패딩 작업
# 가장 긴 샘플 길이 확인

max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이: {}'.format(max_len))

샘플의 최대 길이: 24


In [84]:
sequences = pad_sequences(sequences, maxlen = max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [86]:
# x, y 분리

sequences = np.array(sequences)
x = sequences[:,:-1]
y = sequences[:,-1]

In [87]:
print(x[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [88]:
print(y[:3])

[ 269  371 1115]


In [89]:
# 레이블을 원핫 인코딩

y = to_categorical(y, num_classes = vocab_size)

#### 2) LSTM 모델

In [90]:
from tensorflow.keras.layers import LSTM

In [91]:
model = Sequential()

model.add(Embedding(vocab_size, 10, input_length = max_len - 1))  # vocab_size: 단어 개수, input_length: y를 뺸 나머지 데이터
model.add(LSTM(128))
model.add(Dense(vocab_size, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(x, y, epochs = 200, verbose = 2)

Train on 7803 samples
Epoch 1/200
7803/7803 - 12s - loss: 7.6502 - accuracy: 0.0265
Epoch 2/200
7803/7803 - 9s - loss: 7.1162 - accuracy: 0.0309
Epoch 3/200
7803/7803 - 8s - loss: 6.9783 - accuracy: 0.0334
Epoch 4/200
7803/7803 - 8s - loss: 6.8524 - accuracy: 0.0419
Epoch 5/200
7803/7803 - 9s - loss: 6.7103 - accuracy: 0.0424
Epoch 6/200
7803/7803 - 9s - loss: 6.5460 - accuracy: 0.0483
Epoch 7/200
7803/7803 - 8s - loss: 6.3633 - accuracy: 0.0529
Epoch 8/200
7803/7803 - 7s - loss: 6.1698 - accuracy: 0.0575
Epoch 9/200
7803/7803 - 8s - loss: 5.9690 - accuracy: 0.0633
Epoch 10/200
7803/7803 - 9s - loss: 5.7792 - accuracy: 0.0642
Epoch 11/200
7803/7803 - 9s - loss: 5.5931 - accuracy: 0.0725
Epoch 12/200
7803/7803 - 9s - loss: 5.4168 - accuracy: 0.0756
Epoch 13/200
7803/7803 - 9s - loss: 5.2517 - accuracy: 0.0818
Epoch 14/200
7803/7803 - 9s - loss: 5.0926 - accuracy: 0.0898
Epoch 15/200
7803/7803 - 9s - loss: 4.9428 - accuracy: 0.1001
Epoch 16/200
7803/7803 - 8s - loss: 4.7994 - accuracy: 0

Epoch 133/200
7803/7803 - 9s - loss: 0.3827 - accuracy: 0.9113
Epoch 134/200
7803/7803 - 9s - loss: 0.3664 - accuracy: 0.9134
Epoch 135/200
7803/7803 - 9s - loss: 0.3574 - accuracy: 0.9148
Epoch 136/200
7803/7803 - 9s - loss: 0.3511 - accuracy: 0.9134
Epoch 137/200
7803/7803 - 9s - loss: 0.3470 - accuracy: 0.9145
Epoch 138/200
7803/7803 - 9s - loss: 0.3430 - accuracy: 0.9158
Epoch 139/200
7803/7803 - 9s - loss: 0.3420 - accuracy: 0.9141
Epoch 140/200
7803/7803 - 9s - loss: 0.3375 - accuracy: 0.9158
Epoch 141/200
7803/7803 - 9s - loss: 0.3370 - accuracy: 0.9163
Epoch 142/200
7803/7803 - 9s - loss: 0.3327 - accuracy: 0.9162
Epoch 143/200
7803/7803 - 9s - loss: 0.3338 - accuracy: 0.9141
Epoch 144/200
7803/7803 - 9s - loss: 0.3295 - accuracy: 0.9173
Epoch 145/200
7803/7803 - 9s - loss: 0.3270 - accuracy: 0.9161
Epoch 146/200
7803/7803 - 9s - loss: 0.3220 - accuracy: 0.9149
Epoch 147/200
7803/7803 - 8s - loss: 0.3169 - accuracy: 0.9157
Epoch 148/200
7803/7803 - 9s - loss: 0.3158 - accuracy:

In [92]:
# 문장 생성 함수

def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재단어, 반복횟수(예측횟수)
    init_word = current_word   # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n):  # n번 반복
        encoded = t.texts_to_sequences([current_word])[0]           # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 x(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장
        
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어 인덱스와 동일한 단어가 있다면
                break           # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word         # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [93]:
print(sentence_generation(model, t, 'i', 10))

i want to be rich and im not sorry wish the


In [94]:
print(sentence_generation(model, t, 'how', 10))

how to make facebook more accountable rules for automakers takes assad


### 글자 단위 RNN (Char RNN)

- 입출력 단위를 단어 레벨(word-level)에서 글자 레벨(character-level)로 변경하여 RNN을 구현할 수 있다.
- 입출력 단위를 글자로 변경한 것

#### 1) 데이터 전처리

In [101]:
import urllib.request

urllib.request.urlretrieve('http://www.gutenberg.org/files/11/11-0.txt', filename = '11-0.txt')
f = open('11-0.txt', 'rb')

lines = []
for line in f:          # 데이터를 한 줄씩 읽어
    line = line.strip() # \r, \n 제거
    line = line.lower() # 소문자화
    line = line.decode('ascii', 'ignore') # 바이트 열 제거
    if len(line) > 0:
        lines.append(line)
        
f.close()

In [102]:
lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  you may copy it, give it away or',
 're-use it under the terms of the project gutenberg license included',
 'with this ebook or online at www.gutenberg.org']

In [103]:
# 하나의 문자열로 통합하고, 전체 길이 확인

text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 159612


In [104]:
print(text[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it, g


In [105]:
# 글자 집합 생성

char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)

print('글자 집합의 크기: {}'.format(vocab_size))

글자 집합의 크기: 57


아무리 훈련 코퍼스에 수십만 개 이상 많은 영어 단어가 존재한다고 하더라도, 영어 단어를 표현하기 위해서 글자 집합에 포함되는 글자는 26개의 알파벳뿐이기 때문에 집합의 크기가 현저히 작다.

In [106]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))

print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '@': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56}


In [107]:
index_to_char = {}

for key, value in char_to_index.items():
    index_to_char[value] = key

In [108]:
# text 문자열로부터 다수의 문장 샘플들로 분리

seq_length = 60 # 문장의 길이
n_samples = int(np.floor((len(text)-1) / seq_length)) # 문자열 60등분

print('문장 샘플의 수: 2646')

문장 샘플의 수: 2646


In [117]:
# x, y 분리

train_x = []
train_y = []

for i in range(n_samples):
    x_sample = text[i * seq_length: (i+1) * seq_length]
    x_encoded = [char_to_index[c] for c in x_sample]
    train_x.append(x_encoded)
    
    y_sample = text[i * seq_length + 1: (i + 1) * seq_length + 1] 
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [118]:
print(train_x[0])

[50, 38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31]


In [119]:
print(train_y[0])

[38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31, 44]


In [120]:
# 원핫 인코딩

train_x = to_categorical(train_x)
train_y = to_categorical(train_y)

In [121]:
print('train_X의 크기(shape) : {}'.format(train_x.shape)) 
print('train_y의 크기(shape) : {}'.format(train_y.shape))

train_X의 크기(shape) : (2660, 60, 57)
train_y의 크기(shape) : (2660, 60, 57)


#### 2) LSTM 모델

In [122]:
from tensorflow.keras.layers import TimeDistributed

In [124]:
model = Sequential()

model.add(LSTM(256, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=80, verbose=2)

Train on 2660 samples
Epoch 1/80
2660/2660 - 70s - loss: 3.0758 - accuracy: 0.1820
Epoch 2/80
2660/2660 - 70s - loss: 2.7002 - accuracy: 0.2572
Epoch 3/80
2660/2660 - 72s - loss: 2.3668 - accuracy: 0.3339
Epoch 4/80
2660/2660 - 74s - loss: 2.2244 - accuracy: 0.3687
Epoch 5/80
2660/2660 - 74s - loss: 2.1201 - accuracy: 0.3931
Epoch 6/80
2660/2660 - 75s - loss: 2.0375 - accuracy: 0.4125
Epoch 7/80
2660/2660 - 75s - loss: 1.9643 - accuracy: 0.4319
Epoch 8/80
2660/2660 - 77s - loss: 1.9068 - accuracy: 0.4464
Epoch 9/80
2660/2660 - 77s - loss: 1.8458 - accuracy: 0.4660
Epoch 10/80
2660/2660 - 76s - loss: 1.7943 - accuracy: 0.4787
Epoch 11/80
2660/2660 - 78s - loss: 1.7476 - accuracy: 0.4923
Epoch 12/80
2660/2660 - 78s - loss: 1.7022 - accuracy: 0.5050
Epoch 13/80
2660/2660 - 78s - loss: 1.6616 - accuracy: 0.5156
Epoch 14/80
2660/2660 - 79s - loss: 1.6212 - accuracy: 0.5259
Epoch 15/80
2660/2660 - 79s - loss: 1.5846 - accuracy: 0.5348
Epoch 16/80
2660/2660 - 80s - loss: 1.5488 - accuracy: 0.

In [125]:
# 문장 생성 함수

def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]]     # 랜덤 익덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size)) # (1, length, 55) 크기의 x 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in range(length):
        X[0][i][ix[-1]] = 1  # x[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [129]:
sentence_generation(model, 50)

33 번 글자 c 로 예측을 시작!
can find them. as she said this, she cameWARNING:tensorflow:5 out of the last 82 calls to <function _make_execution_function.<locals>.distributed_function at 0x000001C848AAA048> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/beta/tutorials/eager/tf_function#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


'can find them. as she said this, she came upon a ne'